In [1]:
!pip uninstall -y tensorflow && pip install tensorflow-cpu


Found existing installation: tensorflow 2.18.1
Uninstalling tensorflow-2.18.1:
  Successfully uninstalled tensorflow-2.18.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 MB 1.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 47.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 13.4 MB/s eta 0:00:00:00:01
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-tpu 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.1 which is in

In [2]:
# Only works in TPU-backed environments like Colab or Kaggle
!pip install torch_xla -f https://storage.googleapis.com/tpu-pytorch/wheels/tpuvm/colab.html


Looking in links: https://storage.googleapis.com/tpu-pytorch/wheels/tpuvm/colab.html

[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install transformers diffusers einops torch torchvision


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:

from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
from torch.utils.data import DataLoader
import torchvision.transforms as T
from PIL import Image
import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp



class Flickr8k(Dataset):
    def __init__(self, img_dir, captions_file, split_file, img_size=128, transform=None):
        self.img_dir = img_dir
        self.transform = transform or T.Compose([
            T.Resize((img_size, img_size)),
            T.ToTensor(),
            T.Normalize([0.5]*3, [0.5]*3),
        ])

        self.captions = {}
        vocab = set()
        with open(captions_file, 'r', encoding='utf-8') as f:
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) != 2:
                    continue
                fname, caption = parts
                fname = fname.split('#')[0]
                tokens = caption.lower().strip().split()
                vocab.update(tokens)
                self.captions.setdefault(fname, []).append(tokens)

        self.vocab = {word: idx + 2 for idx, word in enumerate(sorted(vocab))}
        self.vocab['<PAD>'] = 0
        self.vocab['<UNK>'] = 1
        self.inv_vocab = {idx: word for word, idx in self.vocab.items()}

        with open(split_file, 'r') as f:
            all_filenames = [line.strip() for line in f]
        self.image_files = [fname for fname in all_filenames if fname in self.captions]

        print(f"Loaded {len(self.image_files)} image-caption pairs.")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        fname = self.image_files[idx]
        path = os.path.join(self.img_dir, fname)
        img = Image.open(path).convert("RGB")
        img = self.transform(img)
        caption = self.captions[fname][0]
        token_ids = [self.vocab.get(word, 1) for word in caption]
        max_len = 16
        if len(token_ids) < max_len:
            token_ids += [0] * (max_len - len(token_ids))
        else:
            token_ids = token_ids[:max_len]
        return img, torch.tensor(token_ids, dtype=torch.long)


In [5]:
import torch.nn as nn

class SimpleTextEmbed(nn.Module):
            def __init__(self, vocab_size=30522, embed_dim=2048):
                super().__init__()
                self.embed = nn.Embedding(vocab_size, embed_dim)
                self.mlp = nn.Sequential(
                    nn.Linear(embed_dim, embed_dim),
                    nn.GELU(),
                    nn.Linear(embed_dim, embed_dim)
                )

            def forward(self, token_ids):
                return self.mlp(self.embed(token_ids))




In [6]:
from collections import defaultdict
import re

class SimpleTokenizer:
    def __init__(self, captions, max_length=250):
        self.word2idx = {"<pad>": 0, "<unk>": 1}
        self.idx2word = ["<pad>", "<unk>"]
        self.max_length = max_length
        self._build_vocab(captions)

    def _build_vocab(self, captions):
        for cap in captions:
            for word in re.findall(r"\w+", cap.lower()):
                if word not in self.word2idx:
                    self.word2idx[word] = len(self.idx2word)
                    self.idx2word.append(word)

    def encode(self, texts):
        batch_ids = []
        for text in texts:
            tokens = re.findall(r"\w+", text.lower())
            ids = [self.word2idx.get(tok, 1) for tok in tokens]
            ids = ids[:self.max_length]
            ids += [0] * (self.max_length - len(ids))  # pad
            batch_ids.append(ids)
        return torch.tensor(batch_ids, dtype=torch.long)


In [7]:
CAP_FILE = "/kaggle/input/flickr8k/Flickr_Data/Flickr_Data/Flickr_TextData/Flickr8k.token.txt"

IMG_DIR = "/kaggle/input/flickr8k/Flickr_Data/Flickr_Data/Images"
SPLIT_FILE = "/kaggle/input/flickr8k/Flickr_Data/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt"

captions = []
with open(CAP_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        _, caption = line.strip().split('\t')
        captions.append(caption)

tok = SimpleTokenizer(captions)


In [8]:

import torch.nn as nn
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=8192):
        super().__init__()
        pos = torch.arange(0, max_len).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
        pe = torch.zeros(1, max_len, d_model)
        pe[0, :, 0::2] = torch.sin(pos * div)
        pe[0, :, 1::2] = torch.cos(pos * div)
        self.register_buffer("pe", pe)

    def forward(self, x):
        # Dynamically adjust positional encoding to match input sequence length
        seq_len = x.size(1)
        if seq_len > self.pe.size(1):
            raise ValueError(f"Input sequence length ({seq_len}) exceeds maximum positional encoding length ({self.pe.size(1)}).")
        return x + self.pe[:, :seq_len, :]

In [9]:
class GatedSSMBlock(nn.Module):
    def __init__(self, d_model, dropout=0.2):
        super().__init__()
        self.U = nn.Linear(d_model, d_model)
        self.F = nn.Linear(d_model, d_model)
        self.O = nn.Linear(d_model, d_model)
        self.norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        forget_gate = torch.sigmoid(self.F(x))
        update = torch.tanh(self.U(x))
        output_gate = torch.sigmoid(self.O(x))
        h = forget_gate * x + (1 - forget_gate) * update
        return self.dropout(self.norm(output_gate * h))


In [10]:
class CrossHeadRouter(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.attn_weights = nn.Parameter(torch.randn(num_heads, d_model))

    def forward(self, head_outputs):
        # head_outputs: list of (B, T, D)
        stacked = torch.stack(head_outputs, dim=2)  # (B, T, H, D)
        weights = torch.einsum('bthd,hd->bth', stacked, self.attn_weights)  # (B, T, H)
        soft_weights = torch.softmax(weights, dim=-1).unsqueeze(-1)         # (B, T, H, 1)
        fused = (stacked * soft_weights).sum(dim=2)                         # (B, T, D)
        return fused


In [11]:
class TokenWiseFeedForward(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.ffn = nn.Sequential(
            nn.Linear(d_model, d_model * 2),
            nn.GELU(),
            nn.Linear(d_model * 2, d_model)
        )

    def forward(self, x):
        return self.ffn(x)


In [12]:
class ParallelSSMHeads(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.heads = nn.ModuleList([GatedSSMBlock(d_model) for _ in range(num_heads)])
        self.router = CrossHeadRouter(d_model, num_heads)
        self.tokenwise_ffn = TokenWiseFeedForward(d_model)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x):
        # x: (B, T, D)
        head_outputs = [head(x) for head in self.heads]  # Each (B, T, D)
        fused = self.router(head_outputs)                # (B, T, D) after soft alignment
        fused = fused + self.tokenwise_ffn(fused)        # Add local dynamics
        return self.norm(fused)


In [13]:
class MultiHeadSSM(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0
        self.head_dim = d_model // num_heads
        self.heads = nn.ModuleList([GatedSSMBlock(self.head_dim) for _ in range(num_heads)])
        self.output_proj = nn.Linear(d_model, d_model)

    def forward(self, x):
        B, T, D = x.shape
        x = x.view(B, T, len(self.heads), self.head_dim)
        x = torch.stack([head(x[:, :, i, :]) for i, head in enumerate(self.heads)], dim=2)
        x = x.view(B, T, D)
        return self.output_proj(x)


In [14]:
class HybridBlock(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.parallel_ssm = ParallelSSMHeads(d_model, num_heads)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x):
        x_ssm = self.parallel_ssm(x)
        return self.norm(x + x_ssm)


In [15]:
import torch.nn as nn
from einops import rearrange
from torch import einsum



class SSMUNet(nn.Module):
    def __init__(self, d_model=256, depth=4, num_heads=2):
        super().__init__()
        self.init_proj = nn.Conv2d(3, d_model, kernel_size=3, padding=1)
        self.hybrid_blocks = nn.Sequential(*[
            HybridBlock(d_model, num_heads) for _ in range(depth)
        ])
        self.norm = nn.LayerNorm(d_model)
        self.to_rgb = nn.Conv2d(d_model, 3, kernel_size=3, padding=1)

    def forward(self, img, txt_emb):
        B, C, H, W = img.shape
        x = self.init_proj(img)                # [B, d_model, H, W]
        x = rearrange(x, "b c h w -> b (h w) c")
        # OPTIONAL: fuse txt_emb into x tokens (FiLM or linear) per token
        x = self.hybrid_blocks(x)
        x = self.norm(x)
        x = rearrange(x, "b (h w) c -> b c h w", h=H)
        return torch.tanh(self.to_rgb(x))


In [ ]:
from torch_xla.distributed.parallel_loader import MpDeviceLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
from torch.utils.data import DataLoader
from tqdm import tqdm
import json

# Paths
IMG_DIR     = "/kaggle/input/flickr8k/Flickr_Data/Flickr_Data/Images"
CAP_FILE    = "/kaggle/input/flickr8k/Flickr_Data/Flickr_Data/Flickr_TextData/Flickr8k.token.txt"
SPLIT_FILE  = "/kaggle/input/flickr8k/Flickr_Data/Flickr_Data/Flickr_TextData/Flickr_8k.trainImages.txt"

# Training parameters
num_epochs = 100
batch_size = 16

def train():
    device = xm.xla_device()

    # Dataset and DataLoader
    dataset = Flickr8k(IMG_DIR, CAP_FILE, SPLIT_FILE, img_size=128)
    loader = MpDeviceLoader(DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True), device)

    # Tokenizer
    tok = dataset.vocab
    with open("tokenizer_vocab.json", "w") as f:
        json.dump(tok, f)

    # Model & optimizer
    text_encoder = SimpleTextEmbed(vocab_size=len(tok)).to(device)
    text_encoder.train()

    model = SSMUNet().to(device)
    model.train()

    loss_fn = nn.MSELoss()
    optimizer = optim.AdamW(list(model.parameters()) + list(text_encoder.parameters()), lr=1e-3)

    for epoch in range(num_epochs):
        print(f"\n[TPU] Starting epoch {epoch+1}/{num_epochs}")
        pbar = tqdm(enumerate(loader), total=len(loader), desc=f"Epoch {epoch+1}", ncols=100)
        for i, (images, token_ids) in pbar:
            images, token_ids = images.to(device), token_ids.to(device)

            optimizer.zero_grad(set_to_none=True)
            text_emb = text_encoder(token_ids)
            outputs = model(images, text_emb)
            loss = loss_fn(outputs, images)

            loss.backward()
            xm.optimizer_step(optimizer)
            xm.mark_step()

            pbar.set_postfix(loss=loss.item())
            if i % 1000 == 0:
                print(met.metrics_report())

    # Save model and encoder
    xm.save(model.state_dict(), "ssmunet.pth")
    xm.save(text_encoder.state_dict(), "text_encoder.pth")

# Run training
train()


E0000 00:00:1750777442.724402      10 common_lib.cc:621] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:232


Loaded 6000 image-caption pairs.

[TPU] Starting epoch 1/100


Epoch 1:   0%|                                                  | 0/375 [00:12<?, ?it/s, loss=0.604]

Metric: DeviceLockWait
  TotalSamples: 6
  Accumulator: 046ms770.104us
  ValueRate: 004ms710.616us / second
  Rate: 0.486424 / second
  Percentiles: 1%=002.473us; 5%=002.473us; 10%=002.473us; 20%=003.315us; 50%=024.068us; 80%=023ms855.648us; 90%=023ms872.956us; 95%=023ms872.956us; 99%=023ms872.956us
Metric: InputOutputAliasCount
  TotalSamples: 1
  Accumulator: 106.00
  Percentiles: 1%=106.00; 5%=106.00; 10%=106.00; 20%=106.00; 50%=106.00; 80%=106.00; 90%=106.00; 95%=106.00; 99%=106.00
Metric: IrValueTensorToXlaData
  TotalSamples: 111
  Accumulator: 034ms617.242us
  ValueRate: 367ms748.947us / second
  Rate: 1210.96 / second
  Percentiles: 1%=027.020us; 5%=030.584us; 10%=033.690us; 20%=036.473us; 50%=045.134us; 80%=088.307us; 90%=098.808us; 95%=209.521us; 99%=010ms996.747us
Metric: LazyTracing
  TotalSamples: 4017
  Accumulator: 654ms636.707us
  ValueRate: 051ms953.766us / second
  Rate: 98.9753 / second
  Percentiles: 1%=000.425us; 5%=000.526us; 10%=000.579us; 20%=000.671us; 50%=011.

Epoch 1: 100%|██████████████████████████████████████| 375/375 [04:08<00:00,  1.51it/s, loss=0.00202]


...

Epoch 68: 100%|█████████████████████████████████████| 375/375 [03:52<00:00,  1.61it/s, loss=6.61e-6]



[TPU] Starting epoch 69/100


Epoch 69:   0%|                                               | 0/375 [00:01<?, ?it/s, loss=7.68e-6]

Metric: DeviceLockWait
  TotalSamples: 153278
  Accumulator: 02m41s516ms752.454us
  ValueRate: 006ms980.904us / second
  Rate: 9.7034 / second
  Percentiles: 1%=002.094us; 5%=002.720us; 10%=002.908us; 20%=003.197us; 50%=012.073us; 80%=001ms026.656us; 90%=002ms587.731us; 95%=003ms195.178us; 99%=006ms672.283us
Metric: InputOutputAliasCount
  TotalSamples: 5
  Accumulator: 1378.00
  ValueRate: 0.13 / second
  Rate: 0.000487114 / second
  Percentiles: 1%=106.00; 5%=106.00; 10%=106.00; 20%=318.00; 50%=318.00; 80%=318.00; 90%=318.00; 95%=318.00; 99%=318.00
Metric: IrValueTensorToXlaData
  TotalSamples: 111
  Accumulator: 034ms617.242us
  ValueRate: 367ms748.947us / second
  Rate: 1210.96 / second
  Percentiles: 1%=027.020us; 5%=030.584us; 10%=033.690us; 20%=036.473us; 50%=045.134us; 80%=088.307us; 90%=098.808us; 95%=209.521us; 99%=010ms996.747us
Metric: LazyTracing
  TotalSamples: 78977517
  Accumulator: 03h27m45s133ms472.417us
  ValueRate: 812ms910.248us / second
  Rate: 1771.97 / second
  

Epoch 69:  67%|████████████████████████▋            | 250/375 [02:36<01:17,  1.61it/s, loss=8.21e-6]

In [ ]:
# Set device
device = xm.xla_device()

# Initialize tokenizer (reusing existing captions list)
captions = []
with open(CAP_FILE, 'r', encoding='utf-8') as f:
    for line in f:
        _, caption = line.strip().split('\t')
        captions.append(caption)
tok = SimpleTokenizer(captions)

# Reinitialize and load model & text encoder on TPU
text_enc = SimpleTextEmbed().to(device)
text_enc.eval()

model = SSMUNet().to(device)
model.eval()


In [ ]:
@torch.no_grad()
def sample_text2img(prompt, model, text_encoder, tokenizer, img_size=128, steps=50, guidance_scale=7.5):
    model.eval()

    device = xm.xla_device()
    
    # Tokenize and encode prompt
    token_ids = tokenizer.encode([prompt]).to(device)
    uncond_ids = tokenizer.encode([""]).to(device)

    text_emb = text_encoder(token_ids)         # ✅ TPU safe
    uncond_emb = text_encoder(uncond_ids)

    # Start from pure Gaussian noise
    x = torch.randn(1, 3, img_size, img_size, device=device)

    # Noise schedule
    beta = torch.linspace(1e-4, 0.02, steps).to(device)
    alpha = 1 - beta
    alpha_hat = torch.cumprod(alpha, dim=0)

    for t in reversed(range(steps)):
        at = alpha_hat[t]
        bt = beta[t]

        x_in = x.clone()
        txt = text_emb.expand(x.size(0), -1, -1)
        uncond = uncond_emb.expand(x.size(0), -1, -1)

        noise_pred_uncond = model(x_in, uncond)
        noise_pred_cond   = model(x_in, txt)
        noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_cond - noise_pred_uncond)

        noise = torch.randn_like(x) if t > 0 else torch.zeros_like(x)
        x = (1 / torch.sqrt(alpha[t])) * (
            x - (bt / (1 - at).sqrt()) * noise_pred
        ) + noise * bt.sqrt()

    return x.clamp(-1, 1)


In [ ]:
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

prompt = "A mosaic of Alexios Komnenos in battle in the style of 1100s byzantine mosaics."
generated = sample_text2img(prompt, model, text_enc, tok, img_size=384)

# Plot
img = (generated[0].cpu().permute(1, 2, 0).numpy() + 1) / 2  # unnormalize
plt.imshow(img)
plt.title(f"Prompt: {prompt}")
plt.axis("off")
plt.show()
